In [21]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\model\\modelAI.py'>

In [22]:
## parameters
indicatorCodeAI = 'IN3_02'
month = 'ENERO'
month_number = '2023-01'
year = 2023
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------IN3_02------------------------------------------------------


In [23]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf.info)
indicatorIdsOsmosys = subformDf.osmosys_indicator_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicador_ai_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: c7xnpifldnik44oign
parentFormIdAI: cg7ke70ldnik44oih1
indicatorIdAI: cz2rin0lcza9pft40
indicatorIdsOsmosys: [283 291]


In [24]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))



    SELECT
	distinct
    ie.id ie_id, i.id indicador_id, i.code indicator_code, org.acronym, 
		aio.nombre ai_implementer,
		m.month ,
		m.total_execution, can.code canton_code,'EC'||can.code canton_ai_code, can.description canton,
		coalesce(sum(iv.value ),0) total_canton
    FROM
    osmosys.indicator_executions ie 
    INNER JOIN osmosys.indicators i on ie.performance_indicator_id=i.id and ie.state='ACTIVO'
    INNER JOIN osmosys.projects pr on ie.project_id=pr.id
    INNER JOIN osmosys.organizations org on pr.organization_id=org.id
		INNER JOIN ai_integration.osmosys_ai_implementador_organi_2023 aio on org.id=aio.osmosys_id
    INNER JOIN osmosys.quarters q ON ie.id=q.indicator_execution_id and q.state='ACTIVO'
    INNER JOIN osmosys.months m ON q.id=m.quarter_id and m.state='ACTIVO'
		INNER JOIN osmosys.indicator_values iv on m.id=iv.month_id and iv.state='ACTIVO'
		INNER JOIN osmosys.cantones can on iv.canton_id=can.id
    WHERE 
         i.id in (283,291) and
        m.month = 

In [25]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [27]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getTotalMonthByCanton(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    IN3_01 = int(df.iloc[0].value_a)
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[0].value_a

    subform = model.modelAI.SubFormIN3_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        IN3_01=IN3_01,
        IN3_01_CUAL=commentary
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 1


In [9]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: ENERO\data_IN3_02.json


In [10]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------IN3_02------------------------------------------------------
